In [35]:
! pip install feature_engine
from feature_engine.encoding import WoEEncoder

In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

read_train = pd.read_csv('../train.csv')
read_test = pd.read_csv('../test.csv')
sample = pd.read_csv('../sample_submission.csv')

In [37]:
from matplotlib.ticker import MaxNLocator
import seaborn as sns
from cycler import cycler
from IPython.display import display
import math
import os
import random
import gc
import sys
import warnings
warnings.filterwarnings('ignore')

In [38]:
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.impute import KNNImputer
from sklearn import linear_model
from sklearn.linear_model import HuberRegressor

In [39]:
fail = read_train['failure']
read_train.drop('failure',axis=1, inplace = True)
data = pd.concat([read_train, read_test])
data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)

In [40]:
col = [col for col in read_test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
correlation_total = []
selected_col =[]
for x in range(3,18):   
    corr = np.absolute(data.drop(col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
    correlation_total.append(np.round(np.sum(corr[1:5]),5))
    selected_col.append(f'measurement_{x}')
col_feature = pd.DataFrame()
col_feature['Selected columns'] = selected_col
col_feature['correlation total'] = correlation_total

In [41]:
feature = [f for f in read_test.columns if f.startswith('measurement') or f=='loading']
full_fill_dict ={}
full_fill_dict['measurement_17'] = {
    'A': [                'measurement_4','measurement_5','measurement_6','measurement_7','measurement_8'],
    'B': ['measurement_3','measurement_4','measurement_5',                'measurement_7',                'measurement_9'],
    'C': [                                'measurement_5','measurement_6','measurement_7','measurement_8','measurement_9'],
    'D': ['measurement_3',                'measurement_5','measurement_6','measurement_7','measurement_8'],
    'E': [                'measurement_4','measurement_5','measurement_6',                'measurement_8','measurement_9'],
    'F': [                'measurement_4','measurement_5','measurement_6','measurement_7'],
    'G': [                'measurement_4','measurement_5','measurement_6',                'measurement_8','measurement_9'],
    'H': [                'measurement_4','measurement_5',                'measurement_7','measurement_8','measurement_9'],
    'I': ['measurement_3','measurement_4',                'measurement_7','measurement_8','measurement_9']
}

In [42]:
for i in range(8):
    measurement_col = 'measurement_' + col_feature.iloc[i,0][12:]
    fill_dict ={}
    for x in data.product_code.unique() : 
        corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
        measurement_col_dic = {}
        measurement_col_dic[measurement_col] = corr[1:5].index.tolist()
        fill_dict[x] = measurement_col_dic[measurement_col]
    full_fill_dict[measurement_col] =fill_dict
    
feature = [f for f in data.columns if f.startswith('measurement') or f=='loading']
nullValue_cols = [col for col in read_train.columns if read_train[col].isnull().sum()!=0]

In [43]:
for code in data.product_code.unique():
    total_na_filled_by_linear_model = 0
    for measurement_col in list(full_fill_dict.keys()):
        tmp = data[data.product_code==code]
        column = full_fill_dict[measurement_col][code]
        tmp_train = tmp[column+[measurement_col]].dropna(how='any')
        tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]

        model = HuberRegressor(epsilon=1.35, max_iter = 400)
        model.fit(tmp_train[column], tmp_train[measurement_col])
        data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])
        total_na_filled_by_linear_model += len(tmp_test)
    NA = data.loc[data["product_code"] == code,nullValue_cols].isnull().sum().sum()
    model1 = KNNImputer(n_neighbors=3)
    data.loc[data.product_code==code,feature] = model1.fit_transform(data.loc[data.product_code==code, feature])

In [44]:
def _scale(train_data, val_data, test_data, feats):
    scaler = StandardScaler()
    
    scaled_train = scaler.fit_transform(train_data[feats])
    scaled_val = scaler.transform(val_data[feats])
    scaled_test = scaler.transform(test_data[feats])
    
    #back to dataframe
    new_train = train_data.copy()
    new_val = val_data.copy()
    new_test = test_data.copy()
    
    new_train[feats] = scaled_train
    new_val[feats] = scaled_val
    new_test[feats] = scaled_test
    
    assert len(train_data) == len(new_train)
    assert len(val_data) == len(new_val)
    assert len(test_data) == len(new_test)
    
    return new_train, new_val, new_test

In [45]:
le = LabelEncoder()
data['attribute_1'] = le.fit_transform(data['attribute_1'])

read_train = data.iloc[:read_train.shape[0],:]
read_test = data.iloc[read_train.shape[0]:,:]
print(read_train.shape, read_test.shape)

groups = read_train.product_code
X = read_train

woe_encoder = WoEEncoder(variables=['attribute_0'])
woe_encoder.fit(X, fail)
X = woe_encoder.transform(X)
read_test = woe_encoder.transform(read_test)

(26570, 27) (20775, 27)


In [51]:
frame = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13, 14], [15, 16], [17, 18], [19, 20]])
target = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
grouped = np.array([0, 0, 1, 1, 2, 2, 3, 3, 4, 4 ])
DataFrame = pd.DataFrame(frame)
DataFrame['product_code'] = grouped

class ProductSplit():
    def __init__(self):
        self.name = "Splitter in the validation set with two product codes"
        
    def split(self, X: pd.DataFrame, _y=None, _groups=None):
        indices = list(X.groupby("product_code").indices.values())
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                yield [
                    np.concatenate([ix for k, ix in enumerate(indices) if k not in [i, j]]),
                    np.concatenate([ix for k, ix in enumerate(indices) if k in [i, j]])
                    ]

cv = ProductSplit()
for train_index, test_index in cv.split(DataFrame, fail, grouped):
    print("TRAIN:", train_index, "TEST:", test_index)
del cv

In [57]:
imp = []
select_feature = ['loading', 'attribute_0','measurement_17','measurement_0','measurement_1','measurement_2','m3_missing','m5_missing',]
model_res = np.zeros(len(read_test))
x_train, x_val, x_test = _scale(X[select_feature], X[select_feature], read_test, select_feature)
model = linear_model.LogisticRegression(max_iter=200, C=0.0001, penalty='l2', solver='newton-cg')
model.fit(x_train[select_feature], fail)
imp.append(model.coef_.ravel())

model_res += model.predict_proba(x_test[select_feature])[:, 1]
DFrame = pd.DataFrame(np.array(imp).T, index=X[select_feature].columns)
DFrame['mean'] = DFrame.mean(axis=1).abs()
DFrame['feature'] = X[select_feature].columns
DFrame = DFrame.sort_values('mean', ascending=False).reset_index().head(20)
sample['failure'] = model_res
sample.to_csv(f"./submission.csv", index=False)